In [11]:
PROJECT_DIR=`git rev-parse --show-toplevel`
cd $PROJECT_DIR/kmer-mlst

# Get MLST profiles

In [2]:
wget https://pubmlst.org/data/alleles/senterica/aroC.tfa

--2020-01-18 15:57:29--  https://pubmlst.org/data/alleles/senterica/aroC.tfa
Resolving pubmlst.org (pubmlst.org)... 129.67.24.31
Connecting to pubmlst.org (pubmlst.org)|129.67.24.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 511143 (499K)
Saving to: ‘aroC.tfa’

aroC.tfa            100%[===================>] 499.16K   856KB/s    in 0.6s    

2020-01-18 15:57:30 (856 KB/s) - ‘aroC.tfa’ saved [511143/511143]



In [7]:
du -sh aroC.tfa

500K	aroC.tfa


Pick first 10 alleles from this file.

In [12]:
head -n 100 aroC.tfa > aroC-10.fasta

Align everything with clustalomega.

In [58]:
conda run --name clustalo clustalo -i aroC-10.fasta -o aroC-10.aligned.fasta --force
conda run --name clustalo clustalo -i aroC-10.fasta --outfmt phylip -o aroC-10.aligned.phy --force

Also build phylogenetic tree.

In [61]:
conda run --name phyml phyml -i aroC-10.aligned.phy | head -n 3



. Command line: phyml -i aroC-10.aligned.phy 


In [63]:
cat aroC-10.aligned.phy_phyml_tree.txt

(aroC_9:0.00635288,aroC_3:0.00206180,(aroC_6:0.00208228,(aroC_2:0.00000001,(aroC_1:0.00866459,(aroC_10:0.00000001,(aroC_4:0.00000001,(aroC_5:0.00423962,(aroC_8:0.00417796,aroC_7:0.00000001)0.000000:0.00000001)0.962873:0.00207799)0.977224:0.00207798)0.791440:0.00209504)0.791428:0.00209499)0.879313:0.00209523)0.991691:0.00428820);


# Identify k-mers associated with particular SNPs

## First, let's construct simulated reads with particular alleles inserted in them

Make fasta files with a particular mlst allele inserted.

In [108]:
ls test/fastq

08-5578.fasta  real


In [135]:
python -c '
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
for record in SeqIO.parse("aroC-10.fasta", "fasta"):
    ref_record = next(SeqIO.parse("test/fastq/08-5578.fasta", "fasta"))
    mod_record = SeqRecord(Seq(str(ref_record[0:10000].seq) + str(record.seq) + str(ref_record[10000:-1].seq)), 
        id=record.id + "_" + ref_record.id)

    SeqIO.write(mod_record, f"test/fastq/{mod_record.id}.fasta", "fasta")
'

In [138]:
du -sh test/fastq/*.fasta

3.0M	test/fastq/08-5578.fasta
3.0M	test/fastq/aroC_10_NC_013766.fasta
3.0M	test/fastq/aroC_1_NC_013766.fasta
3.0M	test/fastq/aroC_2_NC_013766.fasta
3.0M	test/fastq/aroC_3_NC_013766.fasta
3.0M	test/fastq/aroC_4_NC_013766.fasta
3.0M	test/fastq/aroC_5_NC_013766.fasta
3.0M	test/fastq/aroC_6_NC_013766.fasta
3.0M	test/fastq/aroC_7_NC_013766.fasta
3.0M	test/fastq/aroC_8_NC_013766.fasta
3.0M	test/fastq/aroC_9_NC_013766.fasta


Okay, we've written fasta files with the appropriate allele substituted in. Let's simulate reads.

In [50]:
commands_file=`mktemp`
for input in test/fastq/aroC*.fasta
do
    echo "art_illumina -ss MSv3 -na -l 250 -f 100 -i ${input} -o ${input}.reads --rndSeed $RANDOM > /dev/null" >> ${commands_file}
done

echo "Will execute commands from file ${commands_file}:"
cat ${commands_file}
conda run --name art parallel -j 10 -a ${commands_file}

Will execute commands from file /tmp/tmp.dfSx7AmuU9:
art_illumina -ss MSv3 -na -l 250 -f 100 -i test/fastq/aroC_10_NC_013766.fasta -o test/fastq/aroC_10_NC_013766.fasta.reads --rndSeed 23857 > /dev/null
art_illumina -ss MSv3 -na -l 250 -f 100 -i test/fastq/aroC_1_NC_013766.fasta -o test/fastq/aroC_1_NC_013766.fasta.reads --rndSeed 23147 > /dev/null
art_illumina -ss MSv3 -na -l 250 -f 100 -i test/fastq/aroC_2_NC_013766.fasta -o test/fastq/aroC_2_NC_013766.fasta.reads --rndSeed 12388 > /dev/null
art_illumina -ss MSv3 -na -l 250 -f 100 -i test/fastq/aroC_3_NC_013766.fasta -o test/fastq/aroC_3_NC_013766.fasta.reads --rndSeed 937 > /dev/null
art_illumina -ss MSv3 -na -l 250 -f 100 -i test/fastq/aroC_4_NC_013766.fasta -o test/fastq/aroC_4_NC_013766.fasta.reads --rndSeed 375 > /dev/null
art_illumina -ss MSv3 -na -l 250 -f 100 -i test/fastq/aroC_5_NC_013766.fasta -o test/fastq/aroC_5_NC_013766.fasta.reads --rndSeed 22326 > /dev/null
art_illumina -ss MSv3 -na -l 250 -f 100 -i test/fastq/aroC_6_

In [51]:
du -sh test/fastq/*.fq

614M	test/fastq/aroC_10_NC_013766.fasta.reads.fq
613M	test/fastq/aroC_1_NC_013766.fasta.reads.fq
613M	test/fastq/aroC_2_NC_013766.fasta.reads.fq
613M	test/fastq/aroC_3_NC_013766.fasta.reads.fq
613M	test/fastq/aroC_4_NC_013766.fasta.reads.fq
613M	test/fastq/aroC_5_NC_013766.fasta.reads.fq
613M	test/fastq/aroC_6_NC_013766.fasta.reads.fq
613M	test/fastq/aroC_7_NC_013766.fasta.reads.fq
613M	test/fastq/aroC_8_NC_013766.fasta.reads.fq
613M	test/fastq/aroC_9_NC_013766.fasta.reads.fq


## Now, let's identify SNP-containing kmers

Let's now identify/list SNP-containing kmers from alleles.

In [31]:
python -c '
from Bio import AlignIO
from Bio.Seq import Seq
align = AlignIO.read("aroC-10.aligned.fasta", "fasta")

def check_freq(str):
    freq = {}
    min=1000
    min_char = None
    for c in set(str):
       freq[c] = str.count(c)
       if freq[c] < min:
           min = freq[c]
           min_char = c
    return (freq, min_char)

table_of_kmers = {}
for i in range(align.get_alignment_length()):
    col = align[:, i]
    (cf, min_char) = check_freq(col)
    
    if len(cf) > 1:
        left_edge = max(0, i - 10)
        right_edge = min(align.get_alignment_length(), left_edge + 21)
        kmer_slice = align[:, left_edge:right_edge]
        
        for j in kmer_slice:
            kmer_string = str(j.seq)
            #kmer_seq_obj = Seq(kmer_string)
            #kmer_rc = str(kmer_seq_obj.reverse_complement())
            #kmer_kmer_rc = kmer_string + "|" + kmer_rc
            
            if not j.id in table_of_kmers:
                #table_of_kmers[j.id] = set([kmer_kmer_rc])
                table_of_kmers[j.id] = set([kmer_string])
            else:
                table_of_kmers[j.id].add(kmer_string)
                #table_of_kmers[j.id].add(kmer_kmer_rc)
                
for id in table_of_kmers:
    n = f"{id}.kmers"
    f = open(n, "w")
    print(f"Writing file {n}")
    for kmer in sorted(table_of_kmers[id]):
        f.write(f"{kmer}\n")
    f.close()
'

Writing file aroC_1.kmers
Writing file aroC_2.kmers
Writing file aroC_3.kmers
Writing file aroC_4.kmers
Writing file aroC_5.kmers
Writing file aroC_6.kmers
Writing file aroC_7.kmers
Writing file aroC_8.kmers
Writing file aroC_9.kmers
Writing file aroC_10.kmers


I've written my list of candidate kmers. Let's look at one file.

In [32]:
cat aroC_1.kmers
wc -l aroC_1.kmers

AAAAAGAGGGTGACTCCATCG
AAAAGTTCGGCATCGAAATCC
ACGGCCTGCGCGATTACCGCG
AGATGGGCGACATTCCGCTGG
CAAGAAATACTTGGCGGAAAA
CAGGGGCGATCGCCAAGAAAT
CGTTCTTTTGCCCCGATGCGG
GATTTAACGTGGTGGCGCTGC
GCCCCGATGCGGACAAACTTG
GCGAACCGGTATTTGACCGAC
GCGATTACCGCGGCGGTGGAC
GCTGGACGAACTGATGCGCGC
GGACAAACTTGACGCGCTGGA
GTTTTTCGCCCGGGACACGCG
TGATGAGCATCAATGCGGTGA
TGGCGCTGCGCGGCAGCCAGA
16 aroC_1.kmers


Looks good. Let's check how many k-mers in every file.

In [33]:
wc -l *.kmers

  16 aroC_10.kmers
  16 aroC_1.kmers
  16 aroC_2.kmers
  16 aroC_3.kmers
  16 aroC_4.kmers
  16 aroC_5.kmers
  16 aroC_6.kmers
  16 aroC_7.kmers
  16 aroC_8.kmers
  16 aroC_9.kmers
 160 total


Great. Let's search for files containing all these kmers to try and identify an MLST allele.

## Search for kmers in simulated reads

We do this by using `grep` to search (and print) any kmers in the alleles kmer file. A match only occurs in allele if we find all kmers (that is, the set of kmers found by `grep` has size 16, or equal to the set of all kmers in the alleles file).

In [53]:
temp_dir=`mktemp -d`
echo "Temp dir is ${temp_dir}"
for reads in test/fastq/*.fq
do
    fq_allele_name=`basename $reads`
    kmer_count_part="${temp_dir}/${fq_allele_name}"
    
    # Build commands
    commands_file=`mktemp`
    for allele in *.kmers
    do
        kmer_count_out="${kmer_count_part}_${allele}.count"
        echo "grep -F -f ${allele} ${reads} -o | sort | uniq -c | wc -l > ${kmer_count_out}" >> ${commands_file}
    done
    
    command="parallel -j 10 -a ${commands_file}"
    echo "Working on ${reads} [${command}]"
    ${command}
done

(echo -e "file\tallele";
for allele in *.kmers
do
    count_unique_allele=`wc -l ${allele} | cut -d ' ' -f 1`
    for fastq_name in ${temp_dir}/*${allele}.count
    do
        fq_allele_name=`basename $fastq_name`
        count_unique_fastq=`cat ${fastq_name}`
        if [ "${count_unique_fastq}" -eq "${count_unique_allele}" ]
        then
            echo -e "${fq_allele_name}\t${allele}"
        fi
    done
done) | tee kmer-match-table.tsv | column -s$'\t' -t

Temp dir is /tmp/tmp.8oAr9D3nAC
Working on test/fastq/aroC_10_NC_013766.fasta.reads.fq [parallel -j 10 -a /tmp/tmp.26s1CkZqs6]
Working on test/fastq/aroC_1_NC_013766.fasta.reads.fq [parallel -j 10 -a /tmp/tmp.HE0z9IG1Zs]
Working on test/fastq/aroC_2_NC_013766.fasta.reads.fq [parallel -j 10 -a /tmp/tmp.E5YGAt9kzK]
Working on test/fastq/aroC_3_NC_013766.fasta.reads.fq [parallel -j 10 -a /tmp/tmp.fP6JBpEXHC]
Working on test/fastq/aroC_4_NC_013766.fasta.reads.fq [parallel -j 10 -a /tmp/tmp.f3ZunWd6Jt]
Working on test/fastq/aroC_5_NC_013766.fasta.reads.fq [parallel -j 10 -a /tmp/tmp.Qj8TqRNzbz]
Working on test/fastq/aroC_6_NC_013766.fasta.reads.fq [parallel -j 10 -a /tmp/tmp.7AiQzpD27Y]
Working on test/fastq/aroC_7_NC_013766.fasta.reads.fq [parallel -j 10 -a /tmp/tmp.9e6UcUi6vn]
Working on test/fastq/aroC_8_NC_013766.fasta.reads.fq [parallel -j 10 -a /tmp/tmp.gbnFlFygzg]
Working on test/fastq/aroC_9_NC_013766.fasta.reads.fq [parallel -j 10 -a /tmp/tmp.pKtDQ5xK1L]
file                       